# Usage example on notMNIST

### First download the dataset from http://yaroslavvb.com/upload/notMNIST/notMNIST_small.mat

- [ ] MLD issue "post AKNN" contains more margin experiments to run

In [1]:
import numpy as np, scipy as sp, time, scipy.io, sklearn
import aknn_alg

notMNIST_small = scipy.io.loadmat("notMNIST_small.mat")['images'].reshape(784, 18724)
nmn = (notMNIST_small.T - 255.0/2)/255.0
labels = scipy.io.loadmat("notMNIST_small.mat")['labels'].astype(int)
labels_to_symbols = { 0: 'A', 1: 'B', 2: 'C', 3: 'D', 4: 'E', 5: 'F', 6: 'G', 7: 'H', 8: 'I', 9: 'J' }
labels = np.array([labels_to_symbols[x] for x in labels])

In [ ]:
import importlib
importlib.reload(aknn_alg)

# Calculate list of exact Euclidean nearest neighbors for each point - use more neighbors for less abstaining at a given parameter setting.
itime = time.time()
nbrs_list = aknn_alg.calc_nbrs_exact(nmn, k=1000, use_nndescent=False)
print('Neighbor indices computed. Time:\t {}'.format(time.time() - itime))

Neighbor indices computed. Time:	 8.779023885726929


In [15]:
dim = nmn.shape[1]
t = AnnoyIndex(dim, 'euclidean')  # Length of item vector that will be indexed

In [16]:
%%time

#a = calc_nbrs_pynndescent(raw_data)
from annoy import AnnoyIndex

import random

for i in range(nmn.shape[0]):
    t.add_item(i, nmn[i, :])

t.build(10) # 10 trees
t.save('test.ann')

# ...

CPU times: user 1.77 s, sys: 453 ms, total: 2.23 s
Wall time: 1.66 s


True

In [18]:
u = AnnoyIndex(dim, 'euclidean')
u.load('test.ann') # super fast, will just mmap the file

[1, 1059, 787, 364, 4336, 461, 962, 6438, 1009, 6655, 6822, 7423, 887, 5636, 591, 6960, 6203, 7047, 7142, 6641, 5996, 7336, 6061, 6159, 190, 7425, 4834, 5315, 6942, 7459, 1089, 588, 6220, 373, 7325, 7359, 6670, 485, 721, 834, 5645, 1690, 6393, 7092, 72, 5744, 6365, 6523, 5997, 6329, 7011, 6269, 6802, 7115, 6681, 1514, 5654, 7404, 6997, 6097, 33, 6536, 5893, 7148, 12293, 1602, 1560, 436, 5880, 55, 5824, 6078, 1534, 6025, 6444, 5896, 666, 6480, 6815, 6513, 6242, 5727, 6302, 6089, 6213, 5771, 6056, 6925, 6230, 1612, 7254, 6838, 6, 7306, 1274, 6714, 1176, 1391, 7180, 5704, 4543, 1536, 1249, 970, 6409, 1865, 6875, 7430, 6026, 6906, 1140, 814, 536, 1245, 6085, 7192, 6927, 6605, 4707, 7080, 7307, 6866, 5841, 6155, 7428, 6253, 5245, 920, 561, 7163, 7381, 6908, 6317, 1474, 6002, 14053, 7113, 1838, 5819, 6857, 5922, 10148, 17718, 6436, 7146, 5751, 7462, 81, 5691, 6869, 1196, 174, 413, 6826, 5696, 5887, 6123, 4826, 6237, 6516, 6889, 6340, 6630, 13529, 960, 6407, 136, 527, 6956, 4304, 7082, 603, 5

In [23]:
results = []
for i in range(nmn.shape[0]):
    results.append(u.get_nns_by_item(i, 1000)) # will find the 1000 nearest neighbors

TypeError: an integer is required (got type list)

In [32]:
np.array(results)[:, 1:]

array([[  555,  1210,  1444, ...,   828,   918,   847],
       [ 1059,   787,   364, ...,  1332,  5842,  1819],
       [ 1416,  1633,  1166, ..., 15974,  1180, 12151],
       ...,
       [ 9358,  3742, 14976, ..., 18207, 15929, 17690],
       [18094, 17431, 18692, ..., 17809, 17289, 18671],
       [17794, 18636, 17266, ..., 18612, 18143, 17405]])

In [31]:
nbrs_list

array([[  555,  1444,  1210, ...,  6055,   709,  1381],
       [ 1059,   787,   364, ...,  5060,  6867,  4856],
       [ 1416,  1633,  1166, ...,  8957,  8599, 15470],
       ...,
       [ 9358,  3742, 14976, ...,  8708,  2002,   812],
       [18094, 17431, 18692, ...,  8154, 18327, 16872],
       [17794, 18636, 17266, ..., 18134, 17345, 16994]])

## Make AKNN predictions

In [4]:
itime = time.time()
aknn_predictions = aknn_alg.predict_nn_rule(nbrs_list, labels)
print('AKNN predictions made. Time:\t {}'.format(time.time() - itime))

AKNN predictions made. Time:	 3.195094108581543


In [5]:
import importlib
importlib.reload(aknn_alg)

ref_data = nmn
log_complexity = 1.0
query_data = None
max_k = 1000

# itime = time.time()
if query_data is None:
    query_data = ref_data
nbrs = sklearn.neighbors.NearestNeighbors(n_neighbors=max_k).fit(ref_data)
distances, indices = nbrs.kneighbors(query_data)
distinct_labels = np.unique(labels)
rngarr = np.arange(indices.shape[1])+1
query_nbrs = labels[indices]
fracs_labels = [np.cumsum(query_nbrs == i, axis=1)/rngarr for i in distinct_labels]
# print("Clustering computed. Time: {}".format(time.time() - itime))
thresholds = log_complexity/np.sqrt(np.arange(indices.shape[1]) + 1)
numlabels_predicted = np.add.reduce([f > (thresholds + 1.0/len(distinct_labels)) for f in fracs_labels])
adaptive_k = np.argmax(numlabels_predicted > 0, axis=1)

# pred_labels = np.zeros(fracs_labels[0].shape[0]).astype(str)
# for i in range(fracs_labels[0].shape[0]):
#     if adaptive_k[i] == 0:
#         pred_labels[i] = '?'
#     else:
#         lst = [f[i, adaptive_k[i]] for f in fracs_labels]
#         pred_labels[i] = distinct_labels[np.argmax(lst)]

In [6]:
itime = time.time()
aknn_predictions_new = aknn_alg.aknn_predict(nmn, labels, max_k=1000)
print('AKNN predictions made. Time:\t {}'.format(time.time() - itime))

AKNN predictions made. Time:	 13.257019996643066


## Comparison with k-NN

In [7]:
kvals = [3,5,7,8,10,30,100]
for i in range(len(kvals)):
    knn_predictions = aknn_alg.knn_rule(nbrs_list, labels, k=kvals[i])
    aknn_cov_ndces = aknn_predictions[1] <= kvals[i]
    aknn_cov = np.mean(aknn_cov_ndces)
    aknn_condacc = np.mean((aknn_predictions[0] == labels)[aknn_cov_ndces])
    print('{}-NN accuracy: \t\t{}'.format(kvals[i], np.mean(knn_predictions == labels)))
    print('AKNN accuracy (k <= {}): \t{} \t\t Coverage: \t{}\n'.format(
        kvals[i], aknn_condacc, aknn_cov))
print('Overall AKNN accuracy: {}'.format(np.mean(aknn_predictions[0] == labels)))

3-NN accuracy: 		0.8750267036957915
AKNN accuracy (k <= 3): 	0.9701739850869926 		 Coverage: 	0.838015381328776

5-NN accuracy: 		0.8833048493911557
AKNN accuracy (k <= 5): 	0.9450811565536099 		 Coverage: 	0.9180196539201025

7-NN accuracy: 		0.8836787011322367
AKNN accuracy (k <= 7): 	0.9408167974157822 		 Coverage: 	0.9258705404828028

8-NN accuracy: 		0.8834650715659047
AKNN accuracy (k <= 8): 	0.9362406530053086 		 Coverage: 	0.935644093142491

10-NN accuracy: 		0.8822901089510788
AKNN accuracy (k <= 10): 	0.9322341209133662 		 Coverage: 	0.9425870540482802

30-NN accuracy: 		0.8767891476180303
AKNN accuracy (k <= 30): 	0.9158672400485169 		 Coverage: 	0.9687032685323649

100-NN accuracy: 		0.858577227088229
AKNN accuracy (k <= 100): 	0.9071918180829072 		 Coverage: 	0.9817346720786156

Overall AKNN accuracy: 0.8925977355265969
